In [ ]:
#准备数据-数据预处理-

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
raw_datasets



c:\Users\62453\miniconda3\envs\hf_env\lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [8]:
raw_datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [10]:
label_names = raw_datasets["train"].features["ner_tags"].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [13]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#这里忘了
inputs = tokenizer(raw_datasets["train"][0]["tokens"],is_split_into_words=True)
inputs.tokens()

     

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [18]:
word_ids = inputs.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [ ]:
def align_labels_with_tokens(labels,word_ids):
     new_labels = []
     current_word = None
     for word_id in word_ids:
          if word_id == None:
               new_labels.append(-100)
          elif word_id != current_word:
               current_word = word_id
               label = -100 if word_id == None else labels[word_id]
               new_labels.append(label)
          else:
               #与前一个token是同一个单词
               label = labels[word_id]
               if label % 2 == 1:
                    label += 1
               new_labels.append(label)
     return new_labels

In [25]:
labels = raw_datasets["train"][2]["ner_tags"]
inputs = tokenizer(raw_datasets["train"][2]["tokens"],is_split_into_words=True)
word_ids = inputs.word_ids()
print(inputs.tokens())
new_labels = align_labels_with_tokens(labels,word_ids)
new_labels


['[CLS]', 'BR', '##US', '##SE', '##LS', '1996', '-', '08', '-', '22', '[SEP]']


[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]

In [26]:
def tokenize_and_align_labels(examples):
     tokenizerd_inputs = tokenizer(
          examples["tokens"],
          truncation = True,
          is_split_into_words = True
     )
     all_labels = examples["ner_tags"]
     new_labels = []
     #错误写法 word_ids = tokenized_inputs.word_ids() 
     # word_ids(i)
     for i, labels in enumerate(all_labels):
          #必须在.word_ids()这个函数传入批次才能拿到对应的批次
          word_ids = tokenizerd_inputs.word_ids(batch_index = i)
          new_labels.append(align_labels_with_tokens(labels,word_ids))
     tokenizerd_inputs["labels"] = new_labels
     return tokenizerd_inputs
                            



In [27]:
tokenized_datasets = raw_datasets.map(
     tokenize_and_align_labels,
     batched=True,
     remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [28]:
from transformers import DataCollatorForTokenClassification

datacollator = DataCollatorForTokenClassification(tokenizer= tokenizer)
#bert-base-casd已经写死了，每个单词是对应什么数字，填充符号pad是多少


In [31]:
batch = datacollator([tokenized_datasets["train"][i] for i in range(2)])
batch["input_ids"]

tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [ ]:
#id映射，模型只认识0123，人只认识B-PER
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {v:k for k, v  in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
     model_checkpoint,
     #这里更换了输出头
     num_labels = len(label_names),
     id2label = id2label,
     label2id = label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
     logits, labels = eval_preds
     predictions = np.argmax(logits,axis=-1)

     true_labels = [[label_names[i] for i in label if i != -100] for label in labels]
     #这里有可能会把特殊token预测为0，所以必须看prediction和真是标签
     true_predictions = [[label_names[i] for (i,j) in zip (prediction,label) if j != -100] for (prediction,label) in zip(predictions,labels)]
     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
     return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
     }

In [37]:
from transformers import TrainingArguments

args = TrainingArguments(
     "bert-finetuned-ner-2",
     eval_strategy="epoch",
     save_strategy= "epoch",
     learning_rate=3e-5,
     num_train_epochs=3,
     weight_decay=0.01,
     load_best_model_at_end=True,
     save_total_limit=1,
)

In [38]:
from transformers import Trainer

trainer = Trainer(
     args= args,
     model= model,
     data_collator=datacollator,
     train_dataset=tokenized_datasets["train"],
     eval_dataset=tokenized_datasets["validation"],
     compute_metrics=compute_metrics,
     tokenizer = tokenizer

)
trainer.train()

C:\Users\62453\AppData\Local\Temp\ipykernel_47948\2669839198.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.073700,0.068862,0.906342,0.933187,0.919569,0.980853
2,0.033200,0.066043,0.931675,0.945473,0.938523,0.985136
3,0.018100,0.059633,0.935265,0.950690,0.942914,0.986961


TrainOutput(global_step=5268, training_loss=0.06050381175296815, metrics={'train_runtime': 358.5425, 'train_samples_per_second': 117.484, 'train_steps_per_second': 14.693, 'total_flos': 920771584279074.0, 'train_loss': 0.06050381175296815, 'epoch': 3.0})

In [39]:
#dataloader实现trainer

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
     tokenized_datasets["train"],
     batch_size=8,
     shuffle=True,
     num_workers=4,
     collate_fn=datacollator,
)
eval_dataloader = DataLoader(
     tokenized_datasets["validation"],
     batch_size=8,
     collate_fn=datacollator,
     num_workers=4,
)

In [40]:
model = AutoModelForTokenClassification.from_pretrained(
     model_checkpoint,
     id2label = id2label,
     label2id = label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),lr=3e-5,weight_decay=0.001)

In [42]:
from accelerate import Accelerator
accelerator = Accelerator()
model,train_dataloader,eval_dataloader,optimizer = accelerator.prepare(
     model,train_dataloader,eval_dataloader,optimizer
)

In [43]:
from transformers import get_scheduler

num_train_epochs = 3
num_train_update_steps = len(train_dataloader)
num_training_steps = num_train_epochs * num_train_update_steps

lr_scheduler = get_scheduler(
     "linear",
     optimizer= optimizer,
     num_warmup_steps=0,
     num_training_steps=num_training_steps,
)



In [44]:
def postprogress(predictions,labels):
     #在评估阶段，detach()是从计算图里分离出来，以后他的计算不需要反向传播求导了
     predictions.detach().cpu().clone().numpy()
     labels.detach().cpu().cpu().numpy()

     true_labels = [[label_names[i] for i in label if i != -100] for label in labels]
     true_predictions = [[label_names[i] for (i,j) in zip(prediction,label) if j != -100] for (prediction,label) in zip(predictions,labels)]
     return true_predictions,true_labels


In [47]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
     model.train()
     for batch in train_dataloader:
          outputs = model(**batch)
          loss = outputs.loss
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
     model.eval()
     for batch in eval_dataloader:
          with torch.no_grad():
               outputs = model(**batch)
          predictions = outputs.logits.argmax(dim = -1)
          labels = batch["labels"]
          true_predictions, true_labels = postprogress(predictions,labels)
          metric.add_batch(predictions=true_predictions,references=true_labels)
     result = metric.compute()
     print(
         f"epoch{epoch}:",
         {
              key:result[f"overall_{key}"]
              for key in ["precision","recall","f1","accuracy"]
         }
     )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch0: {'precision': 0.9090165275732286, 'recall': 0.9348704140020195, 'f1': 0.9217622168754668, 'accuracy': 0.983781715429446}
epoch1: {'precision': 0.9287249875971556, 'recall': 0.9451363177381353, 'f1': 0.9368587872216199, 'accuracy': 0.9851209748631307}
epoch2: {'precision': 0.9270781893004115, 'recall': 0.9478290138000673, 'f1': 0.937338770075726, 'accuracy': 0.9859304173779949}
